In [38]:
import visual_behavior_glm.GLM_analysis_tools as gat
import pandas as pd
from multiprocessing import Pool
import numpy as np
from visual_behavior.data_access import loading
pd.set_option('display.max_columns', 500)

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [50]:
versions = [
    '16_events_engaged_L2_optimize_by_session',
    '16_events_disengaged_L2_optimize_by_session',
#     '16_events_all_L2_optimize_by_session'
]

In [59]:
results = []
for version in versions:
    print('loading results for version {}'.format(version))
    results.append(gat.retrieve_results({'glm_version': version}))
results = pd.concat(results)

loading results for version 16_events_engaged_L2_optimize_by_session
loading results for version 16_events_disengaged_L2_optimize_by_session


<ipython-input-59-0b36c127b9af>:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  results = pd.concat(results)


In [60]:
def get_fraction_engaged(bsid):
    try:
        model_outputs = loading.load_behavior_model_outputs(bsid)
        fraction_engaged = model_outputs['engaged'].mean()
    except:
        fraction_engaged = np.nan
    return {'behavior_session_id': bsid, 'fraction_engaged': fraction_engaged}

In [61]:
%%time
with Pool(32) as pool:
    engagement_df = pd.DataFrame(pool.map(get_fraction_engaged, results['behavior_session_id'].drop_duplicates()))

/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/data_access/loading.py:808: UserWarning: no model outputs saved for behavior_session_id: 853266283
  warnings.warn('no model outputs saved for behavior_session_id: {}'.format(behavior_session_id))
/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/data_access/loading.py:808: UserWarning: no model outputs saved for behavior_session_id: 1038793248
  warnings.warn('no model outputs saved for behavior_session_id: {}'.format(behavior_session_id))
/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/data_access/loading.py:808: UserWarning: no model outputs saved for behavior_session_id: 949348713
  warnings.warn('no model outputs saved for behavior_session_id: {}'.format(behavior_session_id))
/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/data_access/loading.py:808: 

CPU times: user 41.6 ms, sys: 250 ms, total: 291 ms
Wall time: 395 ms


In [62]:
engagement_df

,behavior_session_id,fraction_engaged
0,835820832,0.022056
1,853266283,NaN
2,832981154,0.000000
3,949348713,NaN
4,837108110,NaN
5,1038793248,NaN
6,957032492,0.586617
7,876574139,0.983757
8,993267224,0.441226
9,985792625,0.971708


In [77]:
results.groupby(['ophys_experiment_id','glm_version'])[['Full__avg_cv_var_train']].mean().reset_index().merge(
    results[['ophys_experiment_id','behavior_session_id','session_type','glm_version']],
    left_on = ['ophys_experiment_id', 'glm_version'],
    right_on = ['ophys_experiment_id', 'glm_version'],
    how='left'
).merge(
    engaged_df,
    left_on = 'behavior_session_id',
    right_on = 'behavior_session_id',
).drop_duplicates(subset = ['ophys_experiment_id', 'glm_version']).sort_values(by=['session_type','ophys_experiment_id','glm_version'])

,ophys_experiment_id,glm_version,Full__avg_cv_var_train,behavior_session_id,session_type,fraction_engaged
43,837296345,16_events_engaged_L2_optimize_by_session,NaN,837108110,OPHYS_2_images_A_passive,NaN
454,1038848242,16_events_engaged_L2_optimize_by_session,NaN,1038793248,OPHYS_2_images_B_passive,NaN
17,836258957,16_events_disengaged_L2_optimize_by_session,0.012079,835820832,OPHYS_3_images_A,0.022056
30,836258957,16_events_engaged_L2_optimize_by_session,NaN,835820832,OPHYS_3_images_A,0.022056
356,993369861,16_events_disengaged_L2_optimize_by_session,0.057196,993267224,OPHYS_3_images_A,0.441226
405,993369861,16_events_engaged_L2_optimize_by_session,0.055538,993267224,OPHYS_3_images_A,0.441226
248,877057344,16_events_disengaged_L2_optimize_by_session,NaN,876574139,OPHYS_4_images_B,0.983757
262,877057344,16_events_engaged_L2_optimize_by_session,0.029842,876574139,OPHYS_4_images_B,0.983757
332,957759564,16_events_disengaged_L2_optimize_by_session,0.066072,957032492,OPHYS_4_images_B,0.586617
344,957759564,16_events_engaged_L2_optimize_by_session,0.054122,957032492,OPHYS_4_images_B,0.586617
